In [125]:
import pandas as pd
import numpy as np
import pickle as pk

import pickle as pk
import xgboost as xgb
import matplotlib.pyplot as plt

from scipy.stats import gaussian_kde
import h2o
from h2o.automl import H2OAutoML

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [3]:
pd.set_option("display.max_rows", 8)

In [4]:
df = pd.read_pickle("../../data/df.pkl")
#df

### IA Filter

In [5]:
df = df.loc[df.type_bool == True]
df

,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f14,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC
ID,,,,,,,,,,,,,,,,,,,,,
SN10005,"[[10.027000000001863, 4.996, 8.081], [33.94500...","[[0.015000000006693881, 0.2977, 2.206], [18.94...","[[0.0, 1.033, 1.325], [13.015000000006694, 0.4...","[[0.03500000000349246, 1.826, 3.029], [48.875,...","[0.0, 1.0381919191919462, 2.0763838383838924, ...","([10.027000000001863, 10.782565656567513, 11.5...","([0.015000000006693881, 1.0530404040470573, 2....","([0.0, 1.0281717171717606, 2.056343434343521, ...","([0.03500000000349246, 1.0222929292964054, 2.0...","[2.000004411755352, 2.000003547336676, 2.00000...",...,-26.554909,15.768490,16.436792,17.114795,-20.292951,-24.222566,-11.295562,Ia,True,1.08327
SN10007,"[[0.0, 6.266, 10.48], [10.105999999999767, 17....","[[0.03200000000651926, 7.161, 9.313], [2.01600...","[[2.0, 1.24, 2.429], [10.121000000006461, 20.0...","[[0.9770000000062282, 0.6586, 5.329], [15.9930...","[0.0, 1.5136565656566285, 3.027313131313257, 4...","([0.0, 1.5132626262626456, 3.0265252525252913,...","([0.03200000000651926, 1.5451717171782005, 3.0...","([2.0, 3.49314141414145, 4.9862828282829, 6.47...","([0.9770000000062282, 2.480787878794107, 3.984...","[11.341091517194192, 8.812204685737349, 6.9798...",...,-99.877031,21.670852,24.155364,-26.622112,-27.616185,-19.981432,49.299250,Ia,True,0.82331
SN100361,"[[6.957000000002154, 16.02, 23.92], [7.9690000...","[[0.023000000001047738, 4.903, 3.324], [6.9840...","[[0.0, 0.7103, 2.626], [5.9760000000023865, 5....","[[8.055000000000291, 2.567, 2.665], [15.035000...","[0.0, 1.3031111111111275, 2.606222222222255, 3...","([6.957000000002154, 8.15910101010318, 9.36120...","([0.023000000001047738, 1.315494949496019, 2.6...","([0.0, 1.3026767676768058, 2.6053535353536117,...","([8.055000000000291, 9.27674747474778, 10.4984...","[8.233549869515556, 9.464488874691614, 10.4511...",...,102.074783,-95.727930,4.861555,-29.749073,23.442440,5.441237,12.503390,Ia,True,0.90012
SN100471,"[[0.0, 3.79, 4.764], [11.969000000004598, 1.67...","[[0.023999999997613486, 2.202, 3.146], [3.0429...","[[0.008000000001629815, 0.4567, 2.129], [3.004...","[[0.046999999998661224, 6.92, 2.486], [3.07500...","[0.0, 1.4638181818181601, 2.9276363636363203, ...","([0.0, 1.3621414141414037, 2.7242828282828073,...","([0.023999999997613486, 1.4875757575733735, 2....","([0.008000000001629815, 1.3701818181834458, 2....","([0.046999999998661224, 1.3283737373723963, 2....","[5.63245418874034, 5.603687474301877, 5.594769...",...,-40.831628,28.629443,12.801879,-0.555542,13.065768,-21.776767,9.358561,Ia,True,0.92640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN99404,"[[0.0, 6.586, 10.48], [15.980999999999767, 166...","[[22.0, 3.161, 11.98], [40.97700000000623, 15....","[[0.01600000000325963, 8.663, 7.843], [2.0, 1....","[[15.993000000002212, 5.828, 6.542], [29.95700...","[0.0, 1.745343434343478, 3.490686868686956, 5....","([0.0, 1.7352424242424678, 3.4704848484849355,...","([22.0, 23.382101010101024, 24.76420202020205,...","([0.01600000000325963, 1.7511212121244906, 3.4...","([15.993000000002212, 17.57679797980021, 19.16...","[475.9257034984372, 265.54157358525424, 120.22...",...,496.744492,-1269.560018,826.117566,-2224.164801,66.007045,341.497556,-173.911137,Ia,True,0.89529
SN99438,"[[0.0, 2.33, 2.55], [7.894000000000233, 3.805,...","[[0.030999999995401595, 0.006543, 3.683], [7.9...","[[0.014999999999417923, 0.4446, 1.3], [14.0039...","[[0.06199999999807915, 1.497, 4.486], [7.97199...","[0.0, 1.7255757575757058, 3.4511515151514116, ...","([0.0, 1.6948383838383538, 3.3896767676767077,...","([0.030999999995401595, 1.75597979797521, 3.48...","([0.014999999999417923, 1.7298888888882826, 3....","([0.06199999999807915, 1.7869494949475417, 3.5...","[6.789900493579756, 7.110462975693107, 7.57937...",...,-3.768615,-34.417710,32.878026,-33.942007,34.613437,61.324861,-51.210871,Ia,True,0.75966
SN99457,"[[0.0, 10.06, 4.

### Auto ML

In [2]:
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 . connected.


H2O_cluster_uptime:,14 mins 29 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_fmoliveira_xevfw7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.881 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [27]:
X = df.loc[:,'f1':'REDSHIFT_SPEC']
y = df.loc[:,'REDSHIFT_SPEC']

In [8]:
aml = H2OAutoML(max_models=20, seed=1)

In [31]:
aml.train(x=list(df.loc[:,'f1':'f20'].columns), y='REDSHIFT_SPEC', training_frame=h2o.H2OFrame(X.iloc[0:int(len(X)/2)]))

Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


In [32]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201023_063859,0.00830732,0.0911445,0.00830732,0.0703297,0.052618
StackedEnsemble_BestOfFamily_AutoML_20201023_063859,0.00836131,0.0914402,0.00836131,0.0705972,0.052733
GBM_2_AutoML_20201023_063859,0.00850784,0.0922379,0.00850784,0.0713656,0.0532897
GBM_3_AutoML_20201023_063859,0.00864232,0.0929641,0.00864232,0.0716244,0.0536667
GBM_1_AutoML_20201023_063859,0.00878855,0.0937473,0.00878855,0.0728246,0.0543537
GBM_4_AutoML_20201023_063859,0.00886962,0.0941787,0.00886962,0.0723023,0.0543833
GBM_grid__1_AutoML_20201023_063859_model_2,0.00913061,0.0955542,0.00913061,0.074759,0.05547
GBM_grid__1_AutoML_20201023_063859_model_1,0.00916148,0.0957156,0.00916148,0.0750836,0.0554179
DRF_1_AutoML_20201023_063859,0.00922325,0.0960377,0.00922325,0.0747519,0.055537
XRT_1_AutoML_20201023_063859,0.00929942,0.0964335,0.00929942,0.075213,0.0558522


In [33]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20201023_063859

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.0024366639255082928
RMSE: 0.049362576163610956
MAE: 0.03731127461360622
RMSLE: 0.027831669591557865
R^2: 0.9533558850415533
Mean Residual Deviance: 0.0024366639255082928
Null degrees of freedom: 2538
Residual degrees of freedom: 2529
Null deviance: 132.63601876413733
Residual deviance: 6.186689706865556
AIC: -8050.111566850856

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.008307318113965931
RMSE: 0.091144490310528
MAE: 0.07032971242139732
RMSLE: 0.052618049081769064
R^2: 0.8409762228398334
Mean Residual Deviance: 0.008307318113965931
Null degrees of freedom: 2538
Residual degrees of freedom: 2529
Null deviance: 132.8237591820248
Residual deviance: 21.0922806913595
AIC: -4936.010378449608


In [36]:
preds = aml.leader.predict(h2o.H2OFrame(X.iloc[int(len(X)/2):-1].loc[:,'f1':'f20']))

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [52]:
y_pred = preds.as_data_frame()['predict'].tolist()
y_pred

[0.669639699141921,
 0.5952813059804293,
 0.5269688681364885,
 0.5772791553852977,
 0.8458732196375212,
 0.8138956970596479,
 0.7486053165153318,
 0.2738084768180156,
 0.8857316254003152,
 0.7370428235064224,
 0.9317543405805248,
 0.8712391965220992,
 0.5769676800882356,
 0.9717692425207592,
 0.6953286649813151,
 0.8607709386568105,
 0.4126396929910655,
 0.9222804978261014,
 0.3038816267706002,
 0.5157810888479306,
 0.9614697496010316,
 0.8860144063256088,
 0.9217150751668696,
 0.8755331439763824,
 0.9959032900427494,
 0.8386779463880464,
 0.2095063286615618,
 1.0205983587792786,
 0.6407558726198567,
 0.8204097597106726,
 0.7997199094874557,
 0.627211826629168,
 0.9163295582083998,
 0.5940845891361733,
 0.4052160068965936,
 0.8709900015473122,
 0.7805742628376608,
 0.4191370142596929,
 0.4922370711648234,
 0.7354893985727178,
 0.9076002600095732,
 0.2762531497304239,
 0.8183391672777481,
 0.6024423299414621,
 0.8490664314646305,
 0.922668383731072,
 0.5209063478260504,
 0.6580999783394

In [50]:
true_y = list(df.iloc[int(len(X)/2):-1].loc[:,'REDSHIFT_SPEC'])

In [57]:
RMSE = (mean_squared_error(list(df.iloc[int(len(X)/2):-1].loc[:,'REDSHIFT_SPEC']), results))**(1/2)
RMSE

0.090456344218348

In [118]:
h2o.save_model(aml.leader, path = "../../models/h2o_ML_model_leader")

'/home/fmoliveira/Documents/Pessoal/Supernova_Identification_Article/models/h2o_ML_model_leader/StackedEnsemble_AllModels_AutoML_20201023_063859'

In [131]:
pk.dump(aml, open("../../models/h2o_aml.pkl", 'wb'))